In [1]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
import json

In [2]:
from src import sprout_plus
from src import prep_data as prep
import warnings
warnings.filterwarnings("ignore")

# 1. Load files

In [3]:
inputDir = './tutorial/demo/'
outDir = f'{inputDir}/demo_results/'
sc_exp = pd.read_csv(f'{inputDir}/SC_exp.tsv',sep = '\t',header=0,index_col=0)
sc_meta = pd.read_csv(f'{inputDir}/SC_meta.tsv',sep = '\t',header=0,index_col=0)
st_exp = pd.read_csv(f'{inputDir}/ST_exp.tsv',sep = '\t',header=0,index_col=0)
st_coord = pd.read_csv(f'{inputDir}/ST_coord.tsv',sep = '\t',header=0,index_col=0)
st_decon = pd.read_csv(f'{inputDir}/ST_decon.tsv',sep = '\t',header=0,index_col=0)
# TODO run smurf for demo
sc_smurf = sc_exp + np.random.rand(sc_exp.shape[0],sc_exp.shape[1])*10
lr_df = pd.read_csv('./LR/mouse_LR_pairs.txt',sep='\t',header=None)

# 2.Run sprout

In [4]:
species = 'mouse'
st_tp = 'st'

In [5]:
st_coord.index = ['spot_' + str(x) for x in st_coord.index]
st_exp.index = ['spot_' + str(x) for x in st_exp.index]
st_decon.index = ['spot_' + str(x) for x in st_decon.index]
sc_exp.index = ['cell_' + str(x) for x in sc_exp.index]
sc_meta.index = ['cell_' + str(x) for x in sc_meta.index]
sc_smurf.index = ['cell_' + str(x) for x in sc_smurf.index]

In [6]:
sc_adata, st_adata, sc_ref, lr_df = prep.prep_all_adata_merfish(sc_exp = sc_exp, st_exp = st_exp.iloc[:,:500], sc_distribution = sc_exp, 
                                                        sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)

Data clean and scale are done! Single-cell data has 882 genes, spatial data has 500 genes.


In [7]:
# sc_ref = sc_smurf[sc_ref.columns]

In [8]:
from importlib import reload
reload(sprout_plus)

<module 'src.sprout_plus' from '/public/wanwang6/5.Simpute/1.SCC/SpexMod_v11/src/sprout_plus.py'>

In [9]:
obj_spex = sprout_plus.SproutImpute(save_path = outDir, st_adata = st_adata, weight = st_decon, 
                 sc_ref = sc_ref, sc_adata = sc_adata, cell_type_key = 'celltype', lr_df = lr_df, 
                 st_tp = st_tp)
obj_spex.prep()
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0.1, mean_num_per_spot = 10, mode = 'strict', max_rep = 1, repeat_penalty = 10)

Parameters checked!
Getting svg genes
By setting k as 6, each spot has average 5.978835978835979 neighbors.
	 Estimating the cell number in each spot by the deconvolution result.
	 SpexMod selects 500 feature genes.
	 Init solution: max - 1.00,     mean - 0.91,     min - 0.59
	 Swap selection start...
This spot/cell has no L/R genes expressed, select cell only with exp correlation.
	 Swapped solution: max - 1.00,     mean - 0.94,     min - 0.82


In [10]:
import pickle
with open(f'{outDir}/obj_spex.pkl', 'wb') as file:
    pickle.dump(obj_spex, file)

In [11]:
# obj_spex.sc_ref = sc_ref

In [12]:
alpha, beta, gamma, delta, eta = [1, 0.001, 0.001, 0.1, 0.0005]
alter_sc_exp, new_sc_meta = obj_spex.gradient_descent(
                alpha, beta, gamma, delta, eta, 
                init_sc_embed = False,
                iteration = 2, k = 2, W_HVG = 2,
                left_range = 1, right_range = 2, steps = 1, dim = 2)

Running v11 now...
Init sc_coord by affinity embedding...
Avearge shape correlation is: 0.9667940986980015
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calculation done!
Hyperparameters adjusted.
-----Start iteration 0 -----
Avearge shape correlation is: 0.9667940986980015
First-term calculation done!
Second-term calculation done!


Third term calculation done!
Fourth term calculation done!
---0 self.loss4 30.984567402849347 self.GAMMA 0.012013217182594666 self.GAMMA*self.loss4 0.3722243375191724
---In iteration 0, the loss is:loss1:372.22434,loss2:0.00000,loss3:231.83642,loss4:30.98457,loss5:3007.48413.
---In iteration 0, the loss is:loss1:372.22434,loss2:372.22434,loss3:0.37222,loss4:0.37222,loss5:37.22243.
The total loss after iteration 0 is 782.41556.
-----Start iteration 1 -----
Avearge shape correlation is: 0.9867987055767228
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calculation done!
---1 self.loss4 3.285451255138854e+23 self.GAMMA 0.012013217182594666 self.GAMMA*self.loss4 3.9468839470811295e+21
---In iteration 1, the loss is:loss1:6450277461792993973370880.00000,loss2:27866818060090707288260608.00000,loss3:145.12084,loss4:328545125513885424549888.00000,loss5:27866818060669445541462016.00000.
---In iteration 1, the loss is:loss1:64502774617929939733

In [13]:
from src import optimizers
from src.optimizers import *
from src import utils

In [14]:
self = obj_spex
self.sc_knn = findCellKNN(self.st_coord,self.st_tp,self.sc_agg_meta,self.sc_coord,self.K)

In [15]:
def findSpotNeighbor(st_coord,st_tp):
    # old
    all_x = np.sort(list(set(st_coord.iloc[:,0])))
    delta_x = all_x[1] - all_x[0]

    if st_tp == 'visum':
        n_thred = 2*delta_x
        print(f'Visum format, setting threshold as {n_thred}')
    else:
        n_thred = delta_x + 0.001

    st_dist = pd.DataFrame(distance_matrix(st_coord,st_coord),columns = st_coord.index, index = st_coord.index)
    st_dist[(st_dist < n_thred)&(st_dist >0)] = 1
    st_dist[st_dist != 1] = 0
    return st_dist



def findCellKNN(st_coord,st_tp,sc_meta,sc_coord,k): 
    '''
    st_tp = 'visum'
    k = 2
    sc_coord = obj_spex.sc_coord
    '''
    if st_tp == 'slide-seq':
        sc_knn = findCellKNN_slide(sc_meta,sc_coord)
    else:
        sc_knn = {}
        for key in sc_meta.index.tolist():
            sc_knn[key] = []
        # 1. Find neighboring cells from adjacent spot
        # 1.1 assign ST coord to cells
        _, sc_coord_st = sc_prep(st_coord, sc_meta)
        # 1.2 find neighboring cells from adjacent spot
        sc_nn = findSpotNeighbor(sc_coord_st,st_tp)
        # sc_nn = findSpotKNN(sc_coord_st,st_tp)
        # 1.3 calculate real cell-cell distance
        sc_dist = pd.DataFrame(distance_matrix(sc_coord,sc_coord),columns = sc_meta.index, index = sc_meta.index)
        # 2. One hop cross-spot neighbor within a threshold
        # 2.1 Find threshold, i.e., sc_centroid distance
        sc_coord = pd.DataFrame(sc_coord,columns = ['x','y'])
        sc_coord['spot'] = sc_meta['spot'].values
        sc_centroid = sc_coord.groupby('spot').mean()
        sc_centroid, sc_centroid_cells = sc_prep(sc_centroid, sc_meta)
        sc_centroid_cells_dist = pd.DataFrame(distance_matrix(sc_centroid_cells,sc_centroid_cells),columns = sc_meta.index, index = sc_meta.index)  
        # 2.2 Find one hop neighbor between spots
        sc_dist_st_nn = sc_dist * sc_nn.values
        sc_dist_st_nn = sc_dist_st_nn[sc_dist_st_nn != 0]
        # 2.2.1 Apply cell-centroid threshold
        sc_dist_st_nn = sc_dist_st_nn[sc_centroid_cells_dist > sc_dist_st_nn]
        # 2.2.2 Find k-nearest neighbor in each neighbor
        sc_dist_st_nn['spot'] = sc_meta['spot'].values
        sc_knn = sc_dist_st_nn.groupby('spot').apply(apply_nsmallest, k = k, nn_dict = sc_knn)
        sc_knn = sc_knn[sc_knn.keys()[0]]
        # remove no neighbor cells
        empty_keys = [k for k, v in sc_knn.items() if not v]
        for k in empty_keys:
            del sc_knn[k]
    return sc_knn

In [16]:
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)
with open(f'{outDir}/sc_knn.json', 'w') as fp:
    json.dump(obj_spex.sc_knn, fp)
# utils.save_object(obj_spex, f'{outDir}/obj_spex.pkl')
if st_tp != 'slide-seq':
    new_coord = new_sc_meta[['adj_spex_UMAP1','adj_spex_UMAP2']]
    new_coord.to_csv(f'{outDir}/spot_coord_best.tsv',sep = '\t',header=True,index=True)

***

In [17]:
from src import optimizers
from src.cell_selection import *

In [18]:
picked_time = obj_spex.picked_time

In [19]:
new_sc_meta

,sc_id,spot,exp_cor_before,interface_cor_before,mix_cor_before,exp_cor_after,interface_cor_after,mix_cor_after,celltype,UMAP1,...,st_x,st_y,spex_UMAP1,spex_UMAP2,indice,core1,core2,centering_scale_factor,adj_spex_UMAP1,adj_spex_UMAP2
0,cell_1415,spot_91,0.809559,0.338464,0.762450,0.90250,0.467346,0.858984,Other,0.402534,...,3375,11625,-0.305615,5.275952,180,-0.299884,5.153290,2207.494723,3362.349404,11895.774814
1,cell_772,spot_91,0.809559,0.338464,0.762450,0.90250,0.467346,0.858984,Other,0.553855,...,3375,11625,-0.389545,5.106046,180,-0.299884,5.153290,2207.494723,3177.075850,11520.708960
2,cell_381,spot_91,0.809559,0.338464,0.762450,0.90250,0.467346,0.858984,ExcitatoryL2and3,0.810210,...,3375,11625,-0.259854,5.163502,180,-0.299884,5.153290,0.000000,3375.000000,11625.000000
3,cell_760,spot_91,0.809559,0.338464,0.762450,0.90250,0.467346,0.858984,ExcitatoryL2and3,0.450077,...,3375,11625,-0.219300,5.158659,180,-0.299884,5.153290,0.000000,3375.000000,11625.000000
4,cell_1389,spot_91,0.809559,0.338464,0.762450,0.90250,0.467346,0.858984,ExcitatoryL2and3,0.466739,...,3375,11625,-0.368448,4.997866,180,-0.299884,5.153290,2207.494723,3223.645797,11281.901027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793,cell_884,spot_104,0.953852,0.744149,0.932882,0.92386,0.610471,0.892521,Olig,3.857929,...,4125,7125,2.316116,2.928924,7,2.248930,2.983831,2355.147340,4283.233205,6995.686851
1794,cell_1217,spot_104,0.953852,0.744149,0.932882,0.92386,0.610471,0.892521,Olig,3.934330,...,4125,7125,2.316904,2.899620,7,2.248930,2.983831,2355.147340,4285.089560,6926.670987
1795,cell_539,spot_104,0.953852,0.744149,0.932882,0.92386,0.610471,0.892521,Other,3.879679,...,4125,7125,2.246930,3.023515,7,2.248930,2.983831,2355.147340,4120.289485,7218.461802
1796,cell_1420,spot_104,0.953852,0.744149,0.932882,0.92386,0.610471,0.892521,ExcitatoryL5,3.904593,...,4125,7125,2.242262,2.982023,7,2.248930,2.983831,2355.147340,4109.296225,7120.740940


In [20]:
new_sc_meta['exp_cor_before'].mean()

0.9163655409001603

In [21]:
np.mean(new_sc_meta['exp_cor_after'])

0.9374649773209928

In [22]:
# TODO st data是乱序的只有st coord 是正序 没关系吗？

In [23]:
st_exp = obj_spex.st_exp
st_coord = obj_spex.st_coord
st_aff_profile_df = obj_spex.st_aff_profile_df
sc_exp = obj_spex.sc_exp
sc_meta = obj_spex.sc_meta
sum_sc_agg_exp = obj_spex.sum_sc_agg_exp
sc_agg_aff_profile_df = obj_spex.sc_agg_aff_profile_df
init_sc_df = obj_spex.init_sc_df
picked_time = obj_spex.picked_time
lr_df = obj_spex.lr_df
st_tp = obj_spex.st_tp
p = 0.1
T_HALF = 10

In [24]:
st_dist = optimizers.findSpotNeighbor(st_coord,st_tp)
# dict like: 'spot_0': ['spot_1', 'spot_19']
spots_nn_lst = dict(st_dist.apply(lambda x: x.index[x == 1].tolist(), axis=1))
# dict like: 'tp1':['cell_0','cell_1']
tp_idx_dict = get_tp_idx_dict(sc_meta)
new_spot_cell_dict = {}
spot_idx_lst = list(st_exp.index)
result = pd.DataFrame()

In [25]:
for spot in spot_idx_lst:
    '''
    s_: spot exp of st_exp or sc_agg
    _i: indice numerical index of cell_id or spot
    '''
    ########## ST ##########
    # Transform to numerical spot id, subset from csr_matrix
    s_exp = st_exp.loc[spot]
    norm_s_exp = s_exp/np.std(s_exp)
    ########## SC ########## 
    s_sc_agg_sum = sum_sc_agg_exp.loc[spot]
    norm_s_sc_agg_sum = s_sc_agg_sum/np.std(s_sc_agg_sum)
    # generate baseline corr
    max_exp_cor = np.corrcoef(norm_s_exp,norm_s_sc_agg_sum)[0][1]
    # print(f'Baseline cor of spot {spot} is {max_exp_cor}')
    ###### Interface ########
    nn_spot = spots_nn_lst[spot]
    a_ss = st_aff_profile_df.loc[(spot,nn_spot),:]
    a_cc = sc_agg_aff_profile_df.loc[(spot,nn_spot),:]
    corr = np.diag(np.corrcoef(a_ss, a_cc)[:a_ss.shape[0], a_ss.shape[0]:])
    max_aff_cor = corr.mean()
    # print(f'Baseline cor of interface {spot} is {max_aff_cor}')
    max_mix_cor = max_exp_cor*(1-p) + max_aff_cor*p
    # print(f'Baseline mix cor of {spot} is {max_mix_cor}')
    # for each cell in spot
    spot_cell_lst = init_sc_df[init_sc_df['spot'] == spot]['sc_id'].tolist()
    # print(f'orig spot_cell_lst {spot_cell_lst}')
    for i in range(len(spot_cell_lst)):
        cell = spot_cell_lst[i]
        # print(cell)
        spot_cell_lst.remove(cell)
        # print(spot_cell_lst)
        # calculate remain agg exp
        spot_remain_mat = sc_exp.loc[spot_cell_lst]
        remain_exp = np.sum(spot_remain_mat)
        # get candidate cells from the same type
        removed_type = sc_meta.loc[cell]['celltype']
        candi_cell_id = tp_idx_dict[removed_type]
        candi_exp = sc_exp.loc[candi_cell_id]
        # calculate replaced agg for each candidates
        candi_exp_sum = candi_exp + remain_exp
        # [exp cor]
        exp_candi_cor = candi_exp_sum.T.corrwith(s_exp)
        # [interface cor]
        interface_candi_cor = pd.DataFrame()
        for nn_s in nn_spot:
            # nn_s: the neighbor of spot
            a_sn = a_ss.loc[(spot,nn_s)]
            # nn_s_agg_exp: summed exp of init selected cells of nn_s
            nn_s_agg_exp = sum_sc_agg_exp.loc[nn_s]
            # interface between candidate cells and its neighbors regarding aggregated sum
            candi_aff_profile = cal_sc_candi_aff_profile(nn_s_agg_exp, candi_exp_sum, lr_df)
            interface_candi_tmp = candi_aff_profile.T.corrwith(a_sn)
            interface_candi_tmp = pd.DataFrame(interface_candi_tmp,columns=[nn_s])
            interface_candi_cor = pd.concat((interface_candi_cor,interface_candi_tmp),axis = 1)
    #     break
    # break
        interface_candi_cor['mean'] = interface_candi_cor.mean(axis = 1)
        # print(interface_candi_cor.head(5).index)
        # print(exp_candi_cor.head(5).index)
        # TODO del
        prob = half_life_prob(picked_time['count'].values,T_HALF)
        picked_time['prob'] = prob
        cor_df = interface_candi_cor.loc[exp_candi_cor.index,'mean']*p + (1-p)*exp_candi_cor
        adj_cor_df = picked_time.loc[cor_df.index,'prob'] * cor_df
        max_idx = adj_cor_df.idxmax()
        mix_corr = adj_cor_df.loc[adj_cor_df.idxmax()]
        spot_cell_lst.insert(0,max_idx)
        exp_cor = exp_candi_cor.loc[max_idx]
        inter_cor = interface_candi_cor['mean'].loc[max_idx]
        # update cell picked time
        picked_time.loc[cell,'count'] -= 1
        picked_time.loc[max_idx,'count'] += 1
        # print(f'  Change cell {cell} exp_cor is {exp_cor}; inter_cor is {inter_cor}; mix cor of {spot} is {mix_corr}')
        break


KeyError: "['Sst', 'Nts', 'Ptn', 'Sema3c', 'Sema3e', 'Tac1', 'Tac2', 'S100a10', 'Pon2', 'Penk', 'Pdyn'] not in index"

In [ ]:
adj_cor_df = picked_time.loc[cor_df.index,'prob'] * cor_df
max_idx = adj_cor_df.idxmax()
mix_corr = adj_cor_df.loc[adj_cor_df.idxmax()]
spot_cell_lst.insert(0,max_idx)
exp_cor = exp_candi_cor.loc[max_idx]
inter_cor = interface_candi_cor['mean'].loc[max_idx]
picked_time.loc[max_idx,]

count    1.0
prob     1.0
Name: cell_1037, dtype: float64

In [ ]:
picked_time

,count,prob
cell_0,1.0,0.933033
cell_1,1.0,0.933033
cell_2,1.0,0.933033
cell_3,1.0,0.933033
cell_4,1.0,0.933033
...,...,...
cell_1518,1.0,0.933033
cell_1519,1.0,0.933033
cell_1520,1.0,0.933033
cell_1521,1.0,0.933033


In [ ]:
picked_time.loc[max_idx,'count'] += 1

In [ ]:
    tmp = pd.DataFrame(spot_cell_lst,columns = ['sc_id'])
    tmp['spot'] = spot
    tmp['exp_cor_before'] = max_exp_cor
    tmp['interface_cor_before'] = max_aff_cor
    tmp['mix_cor_before'] = max_mix_cor
    tmp['exp_cor_after'] = exp_cor
    tmp['interface_cor_after']= inter_cor
    tmp['mix_cor_after'] = mix_corr
    result = pd.concat((result,tmp))
    new_spot_cell_dict[spot] = spot_cell_lst
    # break
result['celltype'] = sc_meta.loc[result['sc_id']]['celltype'].values
result.index = range(len(result))
result.index = result.index.map(str)

In [ ]:
def reselect_cell(st_exp, st_coord, st_aff_profile_df, sc_exp, sc_meta, sum_sc_agg_exp, sc_agg_aff_profile_df, init_sc_df, lr_df, st_tp, p = 0.1):
    '''
    Reselect cells from sc exp data for higher exp and interface correlation
    No repeat
    Runtime: 20s for each spot with 10 cells; 2s each cell.
    st_exp = obj_spex.st_exp
    st_coord = obj_spex.st_coord
    st_aff_profile_df = obj_spex.st_aff_profile_df
    sc_exp = obj_spex.sc_exp
    sc_meta = obj_spex.sc_meta
    sum_sc_agg_exp = obj_spex.sum_sc_agg_exp
    sc_agg_aff_profile_df = obj_spex.sc_agg_aff_profile_df
    init_sc_df = obj_spex.init_sc_df
    lr_df = obj_spex.lr_df
    st_tp = obj_spex.st_tp
    p = 0.1
    '''
    st_dist = optimizers.findSpotNeighbor(st_coord,st_tp)
    spots_nn_lst = dict(st_dist.apply(lambda x: x.index[x == 1].tolist(), axis=1))
    # 
    tp_idx_dict = get_tp_idx_dict(sc_meta)
    new_spot_cell_dict = {}
    spot_idx_lst = list(st_exp.index)
    result = pd.DataFrame()
    for spot in spot_idx_lst:
        '''
        s_: spot exp of st_exp or sc_agg
        _i: indice numerical index of cell_id or spot
        '''
        # print(f'reselect cells for spot {spot}')
        ########## ST ##########
        # Transform to numerical spot id, subset from csr_matrix
        s_exp = st_exp.loc[spot]
        norm_s_exp = s_exp/np.std(s_exp)
        ########## SC ########## 
        s_sc_agg_sum = sum_sc_agg_exp.loc[spot]
        norm_s_sc_agg_sum = s_sc_agg_sum/np.std(s_sc_agg_sum)
        # generate baseline corr
        max_exp_cor = np.corrcoef(norm_s_exp,norm_s_sc_agg_sum)[0][1]
        # print(f'Baseline cor of spot {spot} is {max_exp_cor}')
        ###### Interface ########
        nn_spot = spots_nn_lst[spot]
        a_ss = st_aff_profile_df.loc[(spot,nn_spot),:]
        a_cc = sc_agg_aff_profile_df.loc[(spot,nn_spot),:]
        corr = np.diag(np.corrcoef(a_ss, a_cc)[:a_ss.shape[0], a_ss.shape[0]:])
        max_aff_cor = corr.mean()
        # print(f'Baseline cor of interface {spot} is {max_aff_cor}')
        max_mix_cor = max_exp_cor*(1-p) + max_aff_cor*p
        # print(f'Baseline mix cor of {spot} is {max_mix_cor}')
        # for each cell in spot
        spot_cell_lst = init_sc_df[init_sc_df['spot'] == spot]['sc_id'].tolist()
        # print(f'orig spot_cell_lst {spot_cell_lst}')
        for i in range(len(spot_cell_lst)):
            cell = spot_cell_lst[i]
            # print(cell)
            spot_cell_lst.remove(cell)
            # print(spot_cell_lst)
            # calculate remain agg exp
            spot_remain_mat = sc_exp.loc[spot_cell_lst]
            remain_exp = np.sum(spot_remain_mat)
            # get candidate cells from the same type
            removed_type = sc_meta.loc[cell]['celltype']
            candi_cell_id = tp_idx_dict[removed_type]
            candi_exp = sc_exp.loc[candi_cell_id]
            # calculate replaced agg for each candidates
            candi_exp_sum = candi_exp + remain_exp
            # [exp cor]
            exp_candi_cor = candi_exp_sum.T.corrwith(s_exp)
            # [interface cor]
            interface_candi_cor = pd.DataFrame()
            for nn_s in nn_spot:
                # nn_s: the neighbor of spot
                a_sn = a_ss.loc[(spot,nn_s)]
                # nn_s_agg_exp: summed exp of init selected cells of nn_s
                nn_s_agg_exp = sum_sc_agg_exp.loc[nn_s]
                # interface between candidate cells and its neighbors regarding aggregated sum
                candi_aff_profile = cal_sc_candi_aff_profile(nn_s_agg_exp, candi_exp_sum, lr_df)
                interface_candi_tmp = candi_aff_profile.T.corrwith(a_sn)
                interface_candi_tmp = pd.DataFrame(interface_candi_tmp,columns=[nn_s])
                interface_candi_cor = pd.concat((interface_candi_cor,interface_candi_tmp),axis = 1)
            interface_candi_cor['mean'] = interface_candi_cor.mean(axis = 1)
            # print(interface_candi_cor.head(5).index)
            # print(exp_candi_cor.head(5).index)
            # TODO del
            cor_df = interface_candi_cor.loc[exp_candi_cor.index,'mean']*p + (1-p)*exp_candi_cor
            max_idx = cor_df.idxmax()
            mix_corr = cor_df.loc[cor_df.idxmax()]
            spot_cell_lst.insert(0,max_idx)
            exp_cor = exp_candi_cor.loc[max_idx]
            inter_cor = interface_candi_cor['mean'].loc[max_idx]
            # print(f'  Change cell {cell} exp_cor is {exp_cor}; inter_cor is {inter_cor}; mix cor of {spot} is {mix_corr}')
            # break
        tmp = pd.DataFrame(spot_cell_lst,columns = ['sc_id'])
        tmp['spot'] = spot
        tmp['exp_cor_before'] = max_exp_cor
        tmp['interface_cor_before'] = max_aff_cor
        tmp['mix_cor_before'] = max_mix_cor
        tmp['exp_cor_after'] = exp_cor
        tmp['interface_cor_after']= inter_cor
        tmp['mix_cor_after'] = mix_corr
        result = pd.concat((result,tmp))
        new_spot_cell_dict[spot] = spot_cell_lst
        # break
    result['celltype'] = sc_meta.loc[result['sc_id']]['celltype'].values
    result.index = range(len(result))
    result.index = result.index.map(str)
    return result


In [ ]:
st_exp = obj_spex.st_exp
sc_knn = obj_spex.sc_knn 
st_aff_profile_df = obj_spex.st_aff_profile_df
alter_sc_exp = obj_spex.alter_sc_exp
sc_meta = obj_spex.sc_agg_meta
spot_cell_dict = obj_spex.spot_cell_dict
lr_df = obj_spex.lr_df

In [ ]:
term4_df, loss_4 = cal_term4(st_exp,sc_knn,st_aff_profile_df,alter_sc_exp,sc_meta,spot_cell_dict,lr_df)

NameError: name 'cal_term4' is not defined

In [ ]:
obj_spex.ALPHA,obj_spex.BETA,obj_spex.GAMMA,obj_spex.DELTA

(0.020253682220164052,
 0.00025403830843444877,
 0.0008631145032363329,
 0.0018113246793395382)

In [ ]:
obj_spex.loss2

2881.439617870177

In [ ]:
def cal_term4(st_exp,sc_knn,st_aff_profile_df,alter_sc_exp,sc_meta,spot_cell_dict,lr_df):
    ''' 
    st_exp = obj_spex.st_exp
    sc_knn = obj_spex.sc_knn 
    st_aff_profile_df = obj_spex.st_aff_profile_df
    alter_sc_exp = obj_spex.alter_sc_exp
    sc_meta = obj_spex.sc_meta
    spot_cell_dict = obj_spex.spot_cell_dict
    lr_df = obj_spex.lr_df
    '''
    # generate knn_df: cell_idx	-> nn_cell_idx
    knn_df = pd.DataFrame(sc_knn.items(), columns=['cell_idx', 'nn_cell_idx'])
    knn_df = knn_df.explode('nn_cell_idx')
    nn_cell_idx = knn_df['nn_cell_idx'].tolist()
    df = sc_meta.loc[nn_cell_idx].copy()
    knn_df['spot'] = df['spot'].values
    term4_LR = pd.DataFrame()
    loss_4 = 0
    n_knn = 0
    #print(knn_df)
    for spot in st_exp.index:
        #print(spot)
        spot_cells = spot_cell_dict[spot]
        # 1. find knn id and its affiliated spot
        spot_knn_df = knn_df[knn_df['cell_idx'].isin(spot_cells)]
        cell_idx = spot_knn_df['cell_idx']
        cell_nn_idx = spot_knn_df['nn_cell_idx']
        n_knn += len(cell_nn_idx)
        #print(spot_knn_df)
        # some spot has no nn for any cell in it.
        if cell_nn_idx.tolist():
            # 2. calculate acc
            tmp_acc = cal_sc_aff_profile(cell_idx, cell_nn_idx, alter_sc_exp, lr_df)
            tmp_acc = tmp_acc.reset_index()
            del tmp_acc[tmp_acc.columns[0]]
            tmp_acc['spot'] = spot_knn_df['spot'].values
            a_cc = tmp_acc.groupby('spot').mean()
            # 3. calculate ass
            a_ss = st_aff_profile_df.loc[(spot,a_cc.index.tolist()),:]
            a_cc_modi = np.sqrt(a_cc/2)
            a_ss_modi = np.sqrt(a_ss/2)
            res_tmp = a_cc_modi - a_ss_modi.values
            loss_tmp = np.sum((res_tmp**2).values)
            loss_4 += loss_tmp
            if np.isnan(loss_tmp):
                print(f'{spot}')
                print(f'acc{a_cc}')
                print(f'ass{a_ss}')
                print(f'a_cc_modi{a_cc_modi}')
                print(f'a_ss_modi{a_ss_modi}')
                print(f'res_tmp{res_tmp}')
            # 4. calculate multiplier
            sum_r = calSumNNRL(alter_sc_exp, spot_knn_df, cell_nn_idx, lr_df[1])
            sum_l = calSumNNRL(alter_sc_exp, spot_knn_df, cell_nn_idx, lr_df[0])
            res_L = sum_r.groupby('cell_idx').apply(multiply_spots,res_tmp = res_tmp)
            res_R = sum_l.groupby('cell_idx').apply(multiply_spots,res_tmp = res_tmp)
            # ? res_R.columns = sum_r.columns
            res_LR = pd.concat([res_L,res_R],axis =1)
            # sum for each adj spot
            ave_res_LR = res_LR.groupby('cell_idx').sum()
            # ave for same gene in LRdb
            res = ave_res_LR.T.groupby('symbol').mean().T
            term4_LR = pd.concat([term4_LR,res],axis =0)
        #break
    loss_4 /= n_knn
    if np.isnan(loss_4):
        print('nananananana')
    term4_df = complete_other_genes(alter_sc_exp, term4_LR)
    return term4_df, loss_4


In [ ]:
def cal_st_cor(st_exp, sum_sc_agg_exp, spot):
    s_exp = st_exp.loc[spot]
    norm_s_exp = s_exp/np.std(s_exp)
    ########## SC ########## 
    norm_s_sc_agg_sum = sum_sc_agg_exp/np.std(sum_sc_agg_exp)
    # generate baseline corr
    max_exp_cor = np.corrcoef(norm_s_exp,norm_s_sc_agg_sum)[0][1]
    print(max_exp_cor)
    return max_exp_cor

In [ ]:
def reselect_cell(st_exp, st_coord, st_aff_profile_df, sc_exp, sc_meta, sum_sc_agg_exp, sc_agg_aff_profile_df, init_sc_df, lr_df, st_tp, p = 0.1):
    '''
    Reselect cells from sc exp data for higher exp and interface correlation
    No repeat
    Runtime: 20s for each spot with 10 cells; 2s each cell.
    '''
    st_dist = optimizers.findSpotNeighbor(st_coord,st_tp)
    spots_nn_lst = dict(st_dist.apply(lambda x: x.index[x == 1].tolist(), axis=1))
    # 
    tp_idx_dict = get_tp_idx_dict(sc_meta)
    new_spot_cell_dict = {}
    spot_idx_lst = list(st_exp.index)
    result = pd.DataFrame()
    print('Selection start...')
    for spot in spot_idx_lst:
        '''
        s_: spot exp of st_exp or sc_agg
        _i: indice numerical index of cell_id or spot
        '''
        # print(f'reselect cells for spot {spot}')
        ########## ST ##########
        # Transform to numerical spot id, subset from csr_matrix
        s_exp = st_exp.loc[spot]
        norm_s_exp = s_exp/np.std(s_exp)
        ########## SC ########## 
        s_sc_agg_sum = sum_sc_agg_exp.loc[spot]
        norm_s_sc_agg_sum = s_sc_agg_sum/np.std(s_sc_agg_sum)
        # generate baseline corr
        max_exp_cor = np.corrcoef(norm_s_exp,norm_s_sc_agg_sum)[0][1]
        # print(f'Baseline cor of spot {spot} is {max_exp_cor}')
        ###### Interface ########
        nn_spot = spots_nn_lst[spot]
        a_ss = st_aff_profile_df.loc[(spot,nn_spot),:]
        a_cc = sc_agg_aff_profile_df.loc[(spot,nn_spot),:]
        corr = np.diag(np.corrcoef(a_ss, a_cc)[:a_ss.shape[0], a_ss.shape[0]:])
        max_aff_cor = corr.mean()
        # print(f'Baseline cor of interface {spot} is {max_aff_cor}')
        max_mix_cor = max_exp_cor*(1-p) + max_aff_cor*p
        # print(f'Baseline mix cor of {spot} is {max_mix_cor}')
        # for each cell in spot
        spot_cell_lst = init_sc_df[init_sc_df['spot'] == spot]['sc_id'].tolist()
        # print(f'orig spot_cell_lst {spot_cell_lst}')
        for i in range(len(spot_cell_lst)):
            cell = spot_cell_lst[i]
            # print(cell)
            spot_cell_lst.remove(cell)
            # print(spot_cell_lst)
            # calculate remain agg exp
            spot_remain_mat = sc_exp.loc[spot_cell_lst]
            remain_exp = np.sum(spot_remain_mat)
            # get candidate cells from the same type
            removed_type = sc_meta.loc[cell]['celltype']
            candi_cell_id = tp_idx_dict[removed_type]
            candi_exp = sc_exp.loc[candi_cell_id]
            # calculate replaced agg for each candidates
            candi_exp_sum = candi_exp + remain_exp
            # [exp cor]
            exp_candi_cor = candi_exp_sum.T.corrwith(s_exp)
            # [interface cor]
            interface_candi_cor = pd.DataFrame()
            for nn_s in nn_spot:
                # nn_s: the neighbor of spot
                a_sn = a_ss.loc[(spot,nn_s)]
                # nn_s_agg_exp: summed exp of init selected cells of nn_s
                nn_s_agg_exp = sum_sc_agg_exp.loc[nn_s]
                # interface between candidate cells and its neighbors regarding aggregated sum
                candi_aff_profile = cal_sc_candi_aff_profile(nn_s_agg_exp, candi_exp_sum, lr_df)
                interface_candi_tmp = candi_aff_profile.T.corrwith(a_sn)
                interface_candi_tmp = pd.DataFrame(interface_candi_tmp,columns=[nn_s])
                interface_candi_cor = pd.concat((interface_candi_cor,interface_candi_tmp),axis = 1)
            interface_candi_cor['mean'] = interface_candi_cor.mean(axis = 1)
            # print(interface_candi_cor.head(5).index)
            # print(exp_candi_cor.head(5).index)
            # TODO del
            cor_df = interface_candi_cor.loc[exp_candi_cor.index,'mean']*p + (1-p)*exp_candi_cor
            max_idx = cor_df.idxmax()
            mix_corr = cor_df.loc[cor_df.idxmax()]
            spot_cell_lst.insert(0,max_idx)
            exp_cor = exp_candi_cor.loc[max_idx]
            inter_cor = interface_candi_cor['mean'].loc[max_idx]
            # print(f'  Change cell {cell} exp_cor is {exp_cor}; inter_cor is {inter_cor}; mix cor of {spot} is {mix_corr}')
            # break
        tmp = pd.DataFrame(spot_cell_lst,columns = ['sc_id'])
        tmp['spot'] = spot
        tmp['exp_cor_before'] = max_exp_cor
        tmp['interface_cor_before'] = max_aff_cor
        tmp['mix_cor_before'] = max_mix_cor
        tmp['exp_cor_after'] = exp_cor
        tmp['interface_cor_after']= inter_cor
        tmp['mix_cor_after'] = mix_corr
        result = pd.concat((result,tmp))
        new_spot_cell_dict[spot] = spot_cell_lst
        # break
    result['celltype'] = sc_meta.loc[result['sc_id']]['celltype'].values
    return new_spot_cell_dict, result